# **Import Modules & Libraries**

In [1]:
import os
import zipfile
import random
import urllib.request
import tensorflow as tf
from pathlib import Path
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop

# **Process Images**

In [2]:
train_dir = './train'
test_dir = './test1'
classes = ['cat', 'dog']

if not os.path.exists(train_dir):
    train_dir_zip = '../input/dogs-vs-cats/train.zip'
    train_zip_ref = zipfile.ZipFile(train_dir_zip, 'r')
    train_zip_ref.extractall()

if not os.path.exists(test_dir):
    test_dir_zip = '../input/dogs-vs-cats/test1.zip'
    test_zip_ref = zipfile.ZipFile(test_dir_zip, 'r')
    test_zip_ref.extractall()

In [3]:
def move_image(source_path, destination_path):
    Path(source_path).rename(destination_path)
    
def create_sub_directories(train_dir):
    for class_ in classes:
        if os.path.exists(os.path.join(train_dir, class_)):
            os.rmdir(os.path.join(train_dir, class_))
        
    all_images = os.listdir(train_dir)
    
    for class_ in classes:
        os.makedirs(os.path.join(train_dir, class_))
        
    for img_file in all_images:
        source_path = os.path.join(train_dir, img_file)
        class_folder = 'cat' if ('cat' in img_file) else 'dog'
        destination_path = os.path.join(train_dir, class_folder, img_file)
        move_image(source_path, destination_path)
    
create_sub_directories(train_dir) # This run only once

In [4]:
print("Number of training cat images -> ", len(os.listdir(train_dir + '/cat')))
print("Number of training dog images -> ", len(os.listdir(train_dir + '/dog')))

In [5]:
def preprocessing_function(img):
    return tf.keras.applications.inception_v3.preprocess_input(img)
                                                            
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                                                      rotation_range=40,
                                                      width_shift_range=0.2,
                                                      height_shift_range=0.2,
                                                      shear_range=0.2,
                                                      zoom_range=0.2,
                                                      horizontal_flip=True,
                                                      fill_mode='nearest',
                                                      validation_split=0.2,
                                                      preprocessing_function=preprocessing_function
                                                        )

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=128,
                                                    class_mode='binary',
                                                    target_size=(150, 150)
                                                    )

# **Image Classification Model**

In [6]:
weights_url = "https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
weights_file = "inception_v3.h5"

In [7]:
urllib.request.urlretrieve(weights_url, weights_file)

In [8]:
pre_trained_model = InceptionV3(
                        input_shape=(150, 150, 3),
                        include_top=False,
                        weights=None
                            )

pre_trained_model.load_weights(weights_file)

# freeze pre-trained layers
for layer in pre_trained_model.layers:
    layer.trainable = False
    
pre_trained_model.summary()

In [9]:
# We Take MIXED7 Layer As the output

last_layer = pre_trained_model.get_layer('mixed7')
last_output = last_layer.output

# Then Add Few Trainerable Layers for Classification

x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dense(1, activation='sigmoid')(x)

model = Model(pre_trained_model.input, x)

In [11]:
# compile the model
model.compile(
            loss='binary_crossentropy',
            metrics=['acc'],
            optimizer=RMSprop(learning_rate=0.0001)
            )

# train the model (adjust the number of epochs from 1 to improve performance)
history = model.fit(
            train_generator,
            epochs=10,
            verbose=1
                )